In [30]:
import numpy as np
import cv2
import dlib
import os
import math

def check_face(faces) :
    if len(faces) == 0 :
        print("zero face detected in ", original_path)
        return -1
    elif len(faces) > 1 :
        print("many faces detected in ", original_path)
        return -2
    return 0

def draw_landmarks(faces, img) :
    for face in faces :
        landmarks = predictor(img, face)
        landmark_list = []
        for p in landmarks.parts():
            landmark_list.append([p.x, p.y])
        landmark_list = np.array(landmark_list)
        
        for i, pt in enumerate(landmark_list[index]):
            pt_pos = (pt[0], pt[1])
            if i==0 :
                point_f = pt_pos
                cv2.circle(img, pt_pos, 2, (255, 0, 0), -1)
            elif i==27 : 
                point_s = pt_pos
                cv2.circle(img, pt_pos, 2, (0, 0, 255), -1)
            elif i==33: 
                point_e = pt_pos
                cv2.circle(img, pt_pos, 2, (0, 0, 255), -1)
            else :
                cv2.circle(img, pt_pos, 2, (0, 255, 0), -1)       
                
    #cv2.imwrite(landmark_path, img_landmark)
    return (point_f, point_s, point_e)

def get_landmarks(faces, img) :
    for face in faces :
        landmarks = predictor(img, face)
        landmark_list = []
        
        for p in landmarks.parts():
            landmark_list.append([p.x, p.y])
        landmark_list = np.array(landmark_list)
        
        face_mid = []
        for i, pt in enumerate(landmark_list[index]):
            pt_pos = (pt[0], pt[1])
            if i == 33 : nose_end = pt_pos
            elif i == 27 : nose_start = pt_pos
            elif i == 16 : face_right = pt_pos
            elif i == 9 : face_mid.append(pt_pos)
            elif i == 8 : face_mid.append(pt_pos)
            elif i == 7 : face_mid.append(pt_pos)
            elif i == 0 : face_left = pt_pos
                
    return (face_left, face_right, nose_start, nose_end, face_mid)
  
def face_alignment(origin_w, origin_h, nose_start, nose_end, img) :
    # vertical alignment
    isRotated = False
    angle = math.degrees(math.atan((nose_start[0] - nose_end[0])/([nose_end[1] - nose_start[1]])))
    
    if abs(angle)>=3 :
        matrix = cv2.getRotationMatrix2D((nose_start[0], nose_start[1]), angle, 1)
        rotated_img = cv2.warpAffine(img, matrix, (origin_w, origin_h))
        isRotated = True
        return (isRotated, rotated_img)
        
    return (isRotated, img)

def face_crop(isRotated, face_right, face_left, nose_start, face_mid, img) :
    mid_jaw_y = max(face_mid[0][1], face_mid[1][1], face_mid[2][1])
    stand_h = mid_jaw_y - nose_start[1]
    stand_w = 2.5 * stand_h / 2
    start_x = nose_start[0] - stand_w
    end_x = nose_start[0] + stand_w
    start_y = nose_start[1] - int(1.2 * stand_h)
    end_y = nose_start[1] + int(1.2 * stand_h)
    
    print("stand_w :", stand_w, " stand_h : ", stand_h, " start_x : ", start_x, " end_x : ", end_x, " start_y : ", start_y, " end_y : ", end_y)
    try:
        ## crop
        cropped_img = img[int(start_y):int(end_y), int(start_x):int(end_x)]
        ## resize
        result_img = cv2.resize(cropped_img, dsize=(256, 256), interpolation = cv2.INTER_AREA)
        return result_img
    except Exception as e :
        print(str(e))
        return img
    

    
ALL = list(range(0, 68))
RIGHT_EYEBROW = list(range(17, 22))
LEFT_EYEBROW = list(range(22, 27))
NOSE = list(range(27, 36))
RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
MOUTH_OUTLINE = list(range(61, 68))
JAWLINE = list(range(0, 17))
index = ALL

## face detector와 landmark predictor 정의
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
## 테두리에 추가되는 픽셀 수 
pixel = 300
"""
original_common_path = "./dataset/pain/"
landmark_common_path = "./dataset/find_landmarks/mody/"
rotate_common_path = "./dataset/rotate_result/mody/"
cropped_common_path = "./dataset/pain_cropped/mody/"
result_common_path = "./dataset/pain_result/"
"""
original_common_path = "./dataset/aberdeen/"
result_common_path = "./dataset/aberdeen/result/"
origin_file_list = os.listdir(original_common_path)

for file in origin_file_list :
    #이미지 파일인지 확인하기
    if '.jpg' not in file : continue

    # dataset의 위치, 저장 위치
    original_path = original_common_path + file
    #landmark_path = landmark_common_path + file
    #rotate_path = rotate_common_path + file
    #cropped_path = cropped_common_path + file
    result_path = result_common_path + file
    print(original_path)
    
    # get image
    origin_img = cv2.imread(original_path)
    # make border
    h, w, c = origin_img.shape
    img = origin_img
    # 이미지 테두리가 있으면 잘라낸다
    #img = origin_img[10:h, 0:w]
    img = cv2.copyMakeBorder(img, pixel, pixel, pixel, pixel, cv2.BORDER_REPLICATE)
    origin_h, origin_w, origin_c = img.shape
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(img_gray, 1)
    
    # find landmarks
    if check_face(faces) < 0 : continue
    
    # to draw circles/dots in image
    # draw_landmarks(img, faces)
    
    # just get landmark locations
    face_left, face_right, nose_start, nose_end, face_mid = get_landmarks(faces, img)
    #cv2.imwrite(landmark_path, img)
    
    # vertical alignment
    isRotated, rotated_img = face_alignment(origin_w, origin_h, nose_start, nose_end, img)
    #cv2.imwrite(rotate_path, rotated_img)
    
    # crop and resize
    # if face rotated -> need new face landmarks
    if isRotated :
        img_gray = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2GRAY)
        faces = detector(img_gray, 1)
        face_left, face_right, nose_start, nose_end, face_mid = get_landmarks(faces, rotated_img)
    
    result = face_crop(isRotated, face_right, face_left, nose_start, face_mid, rotated_img)
    cv2.imwrite(result_path, result)

./dataset/aberdeen/adrian1.jpg
stand_w : 268.75  stand_h :  215  start_x :  241.25  end_x :  778.75  start_y :  275  end_y :  791
./dataset/aberdeen/adrian2.jpg
stand_w : 268.75  stand_h :  215  start_x :  252.25  end_x :  789.75  start_y :  287  end_y :  803
./dataset/aberdeen/adrian3.jpg
stand_w : 273.75  stand_h :  219  start_x :  234.25  end_x :  781.75  start_y :  272  end_y :  796
./dataset/aberdeen/alec1.jpg
stand_w : 235.0  stand_h :  188  start_x :  283.0  end_x :  753.0  start_y :  305  end_y :  755
./dataset/aberdeen/alison1.jpg
stand_w : 226.25  stand_h :  181  start_x :  314.75  end_x :  767.25  start_y :  337  end_y :  771
./dataset/aberdeen/alison2.jpg
stand_w : 223.75  stand_h :  179  start_x :  314.25  end_x :  761.75  start_y :  316  end_y :  744
./dataset/aberdeen/alison3.jpg
stand_w : 232.5  stand_h :  186  start_x :  298.5  end_x :  763.5  start_y :  309  end_y :  755
./dataset/aberdeen/alister1.jpg
stand_w : 252.5  stand_h :  202  start_x :  269.5  end_x :  774.5 

stand_w : 235.0  stand_h :  188  start_x :  235.0  end_x :  705.0  start_y :  340  end_y :  790
./dataset/aberdeen/catherine17.jpg
stand_w : 231.25  stand_h :  185  start_x :  262.75  end_x :  725.25  start_y :  354  end_y :  798
./dataset/aberdeen/catherine18.jpg
stand_w : 248.75  stand_h :  199  start_x :  246.25  end_x :  743.75  start_y :  326  end_y :  802
./dataset/aberdeen/catherine2.jpg
stand_w : 226.25  stand_h :  181  start_x :  251.75  end_x :  704.25  start_y :  310  end_y :  744
./dataset/aberdeen/catherine3.jpg
stand_w : 227.5  stand_h :  182  start_x :  266.5  end_x :  721.5  start_y :  310  end_y :  746
./dataset/aberdeen/catherine4.jpg
stand_w : 227.5  stand_h :  182  start_x :  253.5  end_x :  708.5  start_y :  309  end_y :  745
./dataset/aberdeen/catherine5.jpg
stand_w : 222.5  stand_h :  178  start_x :  261.5  end_x :  706.5  start_y :  316  end_y :  742
./dataset/aberdeen/catherine6.jpg
stand_w : 227.5  stand_h :  182  start_x :  252.5  end_x :  707.5  start_y :  3

stand_w : 255.0  stand_h :  204  start_x :  259.0  end_x :  769.0  start_y :  286  end_y :  774
./dataset/aberdeen/dpearson11.jpg
stand_w : 256.25  stand_h :  205  start_x :  232.75  end_x :  745.25  start_y :  282  end_y :  774
./dataset/aberdeen/dpearson12.jpg
stand_w : 261.25  stand_h :  209  start_x :  240.75  end_x :  763.25  start_y :  273  end_y :  773
./dataset/aberdeen/dpearson2.jpg
stand_w : 253.75  stand_h :  203  start_x :  253.25  end_x :  760.75  start_y :  278  end_y :  764
./dataset/aberdeen/dpearson3.jpg
stand_w : 253.75  stand_h :  203  start_x :  253.25  end_x :  760.75  start_y :  278  end_y :  764
./dataset/aberdeen/dpearson4.jpg
stand_w : 251.25  stand_h :  201  start_x :  246.75  end_x :  749.25  start_y :  294  end_y :  776
./dataset/aberdeen/dpearson5.jpg
stand_w : 256.25  stand_h :  205  start_x :  251.75  end_x :  764.25  start_y :  279  end_y :  771
./dataset/aberdeen/dpearson6.jpg
stand_w : 256.25  stand_h :  205  start_x :  247.75  end_x :  760.25  start_y

stand_w : 243.75  stand_h :  195  start_x :  252.25  end_x :  739.75  start_y :  310  end_y :  778
./dataset/aberdeen/iroy6.jpg
stand_w : 245.0  stand_h :  196  start_x :  251.0  end_x :  741.0  start_y :  328  end_y :  798
./dataset/aberdeen/itaylor1.jpg
stand_w : 260.0  stand_h :  208  start_x :  264.0  end_x :  784.0  start_y :  293  end_y :  791
./dataset/aberdeen/itaylor2.jpg
stand_w : 263.75  stand_h :  211  start_x :  243.25  end_x :  770.75  start_y :  280  end_y :  786
./dataset/aberdeen/itaylor3.jpg
stand_w : 263.75  stand_h :  211  start_x :  239.25  end_x :  766.75  start_y :  294  end_y :  800
./dataset/aberdeen/jenni1.jpg
stand_w : 256.25  stand_h :  205  start_x :  329.75  end_x :  842.25  start_y :  265  end_y :  757
./dataset/aberdeen/jenni11.jpg
stand_w : 261.25  stand_h :  209  start_x :  333.75  end_x :  856.25  start_y :  270  end_y :  770
./dataset/aberdeen/jenni12.jpg
stand_w : 261.25  stand_h :  209  start_x :  348.75  end_x :  871.25  start_y :  277  end_y :  7

stand_w : 235.0  stand_h :  188  start_x :  296.0  end_x :  766.0  start_y :  284  end_y :  734
./dataset/aberdeen/kirsty12.jpg
stand_w : 236.25  stand_h :  189  start_x :  296.75  end_x :  769.25  start_y :  287  end_y :  739
./dataset/aberdeen/kirsty15.jpg
stand_w : 265.0  stand_h :  212  start_x :  256.0  end_x :  786.0  start_y :  298  end_y :  806
./dataset/aberdeen/kirsty16.jpg
stand_w : 291.25  stand_h :  233  start_x :  240.75  end_x :  823.25  start_y :  280  end_y :  838
./dataset/aberdeen/kirsty17.jpg
stand_w : 267.5  stand_h :  214  start_x :  259.5  end_x :  794.5  start_y :  298  end_y :  810
./dataset/aberdeen/kirsty18.jpg
stand_w : 295.0  stand_h :  236  start_x :  257.0  end_x :  847.0  start_y :  291  end_y :  857
./dataset/aberdeen/kirsty2.jpg
stand_w : 253.75  stand_h :  203  start_x :  278.25  end_x :  785.75  start_y :  278  end_y :  764
./dataset/aberdeen/kirsty3.jpg
stand_w : 256.25  stand_h :  205  start_x :  259.75  end_x :  772.25  start_y :  274  end_y :  76

stand_w : 237.5  stand_h :  190  start_x :  285.5  end_x :  760.5  start_y :  330  end_y :  786
./dataset/aberdeen/martin6.jpg
stand_w : 240.0  stand_h :  192  start_x :  299.0  end_x :  779.0  start_y :  322  end_y :  782
./dataset/aberdeen/martin7.jpg
stand_w : 235.0  stand_h :  188  start_x :  290.0  end_x :  760.0  start_y :  331  end_y :  781
./dataset/aberdeen/martin8.jpg
stand_w : 237.5  stand_h :  190  start_x :  278.5  end_x :  753.5  start_y :  333  end_y :  789
./dataset/aberdeen/martin9.jpg
stand_w : 240.0  stand_h :  192  start_x :  260.0  end_x :  740.0  start_y :  332  end_y :  792
./dataset/aberdeen/meggan1.jpg
stand_w : 257.5  stand_h :  206  start_x :  286.5  end_x :  801.5  start_y :  339  end_y :  833
./dataset/aberdeen/merilyn1.jpg
stand_w : 252.5  stand_h :  202  start_x :  255.5  end_x :  760.5  start_y :  281  end_y :  765
./dataset/aberdeen/michael1.jpg
stand_w : 265.0  stand_h :  212  start_x :  234.0  end_x :  764.0  start_y :  275  end_y :  783
./dataset/abe

stand_w : 266.25  stand_h :  213  start_x :  252.75  end_x :  785.25  start_y :  286  end_y :  796
./dataset/aberdeen/pat2.jpg
stand_w : 248.75  stand_h :  199  start_x :  258.25  end_x :  755.75  start_y :  271  end_y :  747
./dataset/aberdeen/pat3.jpg
stand_w : 248.75  stand_h :  199  start_x :  245.25  end_x :  742.75  start_y :  281  end_y :  757
./dataset/aberdeen/pat4.jpg
stand_w : 248.75  stand_h :  199  start_x :  247.25  end_x :  744.75  start_y :  276  end_y :  752
./dataset/aberdeen/pat5.jpg
stand_w : 253.75  stand_h :  203  start_x :  255.25  end_x :  762.75  start_y :  267  end_y :  753
./dataset/aberdeen/pat6.jpg
stand_w : 253.75  stand_h :  203  start_x :  249.25  end_x :  756.75  start_y :  272  end_y :  758
./dataset/aberdeen/pat7.jpg
stand_w : 250.0  stand_h :  200  start_x :  242.0  end_x :  742.0  start_y :  260  end_y :  740
./dataset/aberdeen/paul!22.jpg
stand_w : 266.25  stand_h :  213  start_x :  160.75  end_x :  693.25  start_y :  294  end_y :  804
./dataset/ab

stand_w : 262.5  stand_h :  210  start_x :  253.5  end_x :  778.5  start_y :  322  end_y :  826
./dataset/aberdeen/tock!22.jpg
stand_w : 285.0  stand_h :  228  start_x :  176.0  end_x :  746.0  start_y :  237  end_y :  783
./dataset/aberdeen/tock1.jpg
stand_w : 277.5  stand_h :  222  start_x :  241.5  end_x :  796.5  start_y :  267  end_y :  799
./dataset/aberdeen/tock11.jpg
stand_w : 288.75  stand_h :  231  start_x :  216.25  end_x :  793.75  start_y :  270  end_y :  824
./dataset/aberdeen/tock12.jpg
stand_w : 283.75  stand_h :  227  start_x :  236.25  end_x :  803.75  start_y :  255  end_y :  799
./dataset/aberdeen/tock15.jpg
stand_w : 308.75  stand_h :  247  start_x :  196.25  end_x :  813.75  start_y :  233  end_y :  825
./dataset/aberdeen/tock16.jpg
stand_w : 312.5  stand_h :  250  start_x :  194.5  end_x :  819.5  start_y :  234  end_y :  834
./dataset/aberdeen/tock17.jpg
stand_w : 300.0  stand_h :  240  start_x :  191.0  end_x :  791.0  start_y :  225  end_y :  801
./dataset/abe